In [1]:
import pandas as pd
import numpy as np

pd.set_option('precision', 13)
pd.options.display.max_rows = 100

In [ ]:
# загружаем города и поселки

all_cities = pd.read_csv('C:/00_Projects/Lenin/01_FIAS_csv_original/03_all_cities.csv')

lenin_cities = pd.read_csv('C:/00_Projects/Lenin/01_FIAS_csv_original/04_lenin_cities.csv')

In [ ]:
# загружаем улицы

all_streets = pd.read_csv('C:/00_Projects/Lenin/01_FIAS_csv_original/05_all_streets.csv')

lenin_streets = pd.read_csv('C:/00_Projects/Lenin/01_FIAS_csv_original/06_lenin_streets.csv')

### Проверяем пустые значения ###

Мы не зря взяли formalname:

In [ ]:
# смотрим пустые значения

all_cities[all_cities.offname.isnull()]

In [ ]:
# в столбце city (бывшем formalname)

all_cities[all_cities.city.isnull()]

**Пропуски в lenin_streets:**

In [ ]:
lenin_streets[lenin_streets.offname.isnull()]

In [ ]:
lenin_streets[lenin_streets.street.isnull()]

In [ ]:
# с city и mun_district такая же история

lenin_streets[lenin_streets.city.isnull()].shape

### Пропуски в all_streets: ###

In [ ]:
all_streets[all_streets.offname.isnull()]

In [ ]:
# в street -- нет

all_streets[all_streets.street.isnull()]

In [ ]:
# есть небольшое количество в city и mun_district

all_streets[all_streets.mun_district.isnull()]

Эти ошибки можно поправить в Эксель "в ручном режиме"

## Заполняем пропуски ##

In [ ]:
# Создаем файл с пустыми значениями

temp = all_streets[(
    (all_streets.mun_district.isnull()) |
    (all_streets.city.isnull()) |
    (all_streets.city_type.isnull())
)]

In [ ]:
# загружаем файл с кодами ОКТМО
#отсюда: https://rosstat.gov.ru/opendata/7708234640-oktmo

oktmo = pd.read_excel('C:/00_Projects/Lenin/Datasets/03_Other/oktmo.xlsx')

In [ ]:
# объединяем с temp

temp = temp.merge(oktmo, on='oktmo', how='left')

In [ ]:
temp.shape

In [ ]:
# выгружаем в Эксель, чтобы дочистить руками (потом автоматизировать)

#temp.to_excel('C:/00_Projects/Lenin/Datasets/temp1.xlsx', encoding='utf-8', index=False)

In [ ]:
# загружаем очищенные данные

temp = pd.read_excel('C:/00_Projects/Lenin/Datasets/temp.xlsx')

In [ ]:
temp.shape

In [ ]:
# удаляем строки с пустыми значениями из основного датафрейма

all_streets = all_streets[~(
    (all_streets.mun_district.isnull()) |
    (all_streets.city.isnull()) |
    (all_streets.city_type.isnull())
)]

In [ ]:
all_streets.shape

In [ ]:
all_streets.columns

In [ ]:
all_streets = all_streets[['aolevel', 'shortname', 'street',
                           'city_type', 'city', 'mun_district', 'region',
                           'okato', 'oktmo', 'postalcode']]

In [ ]:
temp = temp[['aolevel', 'shortname', 'street',
                           'city_type', 'city', 'mun_district', 'region',
                           'okato', 'oktmo', 'postalcode']]

In [ ]:
# присоединяем очищенные данные к основному датафрейму

all_streets = all_streets.append(temp, ignore_index = True)

In [ ]:
all_streets.head()

In [ ]:
sev = pd.read_csv('C:/00_Projects/Lenin/Datasets/05_all_streets_old.csv')

In [ ]:
sev = sev[(sev.region == 'Севастополь')]

In [ ]:
sev.shape

In [ ]:
# отвалился Севастополь по дороге (: Присоединяем

all_streets = all_streets.append(sev, ignore_index = True)

## Удаляем дубликаты ##

Хотя мы удаляли дубликаты на предыдущем этапе, они все равно есть.

Но на всякий случай проверим, точно ли это идентичные записи.

**P.S.** У меня есть гипотеза, что это записи из разных таблиц. Т.е. некоторые записи дублируют в другой регион по ошибке или населенный пункт меняет принадлежность, но остается в таблице бывшего родительского региона с новым кодом.

In [ ]:
# заводим тестовый датафрейм

test = all_cities[all_cities.duplicated(keep=False)]

In [ ]:
# смотрим индексы совпадающих строк

print(test.groupby(test.columns.tolist()).apply(lambda x: x.index.tolist()).values.tolist())

In [ ]:
all_cities.loc[[98418, 98780]]

In [ ]:
all_cities.loc[[1635, 5773]]

То же самое для улиц (на всякий случай, хотя Питон никогде не врет).

In [ ]:
test = lenin_streets[lenin_streets.duplicated(keep=False)]

In [ ]:
# смотрим индексы совпадающих строк

print(test.groupby(test.columns.tolist()).apply(lambda x: x.index.tolist()).values.tolist())

In [ ]:
lenin_streets.loc[[10870, 10879, 10880]]

In [ ]:
lenin_streets.loc[[6389, 6408]]

### Удаляем ###

In [ ]:
all_cities.shape

In [ ]:
all_cities = all_cities.drop_duplicates()

In [ ]:
all_cities.shape

In [ ]:
lenin_cities.shape

In [ ]:
lenin_cities = lenin_cities.drop_duplicates()

In [ ]:
lenin_cities.shape

In [ ]:
all_streets.shape

In [ ]:
all_streets = all_streets.drop_duplicates()

In [ ]:
all_streets.shape

In [ ]:
lenin_streets.shape

In [ ]:
lenin_streets = lenin_streets.drop_duplicates()

In [ ]:
lenin_streets.shape

### Записываем в файлы ###

В папку 01_FIAS_csv_post, на всякий случай.

In [ ]:
all_cities.to_csv('C:/00_Projects/Lenin/01_FIAS_csv_post/03_all_cities.csv',
                  sep=',', encoding='utf-8', index=False)

In [ ]:
lenin_cities.to_csv('C:/00_Projects/Lenin/01_FIAS_csv_post/04_lenin_cities.csv',
                    sep=',', encoding='utf-8', index=False)

In [ ]:
all_streets.to_csv('C:/00_Projects/Lenin/01_FIAS_csv_post/05_all_streets.csv',
                   sep=',', encoding='utf-8', index=False)

In [ ]:
lenin_streets.to_csv('C:/00_Projects/Lenin/01_FIAS_csv_post/06_lenin_streets.csv',
                                  sep=',', encoding='utf-8', index=False)